In [1]:
import random
import numpy as np

def ispisi(konfiguracija):
    n = len(konfiguracija)
    for i in range(n):
        for j in range(n):
            if konfiguracija[i][1] == j + 1:
                 print('D ', end='')
            else:
                print('- ', end='')
        print()

# Za prosledjenu konfiguraciju, racuna broj prekrsaja. Dve dame koje se napadaju
# cine jedan prekrsaj. Takodje, ako se dve dame napadaju "preko" trece dame,
# to se takodje ubraja u prekrsaj, jer bi se uklanjanjem srednje dame one
# napadale. Ne moramo proveravati prekrsaje po vertikalama, jer znamo da je
# na svakoj vertikali tacno jedna dama.
def izracunaj_broj_prekrsaja(konfiguracija):
    n = len(konfiguracija)
    brojac = 0
    for i in range(n):
        for j in range(i + 1, n):
            # Ako se dve dame nalaze na istoj horizontali, to je prekrsaj.
            if konfiguracija[i][1] == konfiguracija[j][1]:
                 brojac += 1
            # Provera dijagonala paralelnih glavnoj dijagonali.
            if konfiguracija[i][1] - (i+1) == konfiguracija[j][1] - (j+1):
                brojac += 1
            # Provera dijagonala paralelnih sporednoj dijagonali.
            if konfiguracija[i][1] + (i+1) == konfiguracija[j][1] + (j+1):
                brojac += 1
    return brojac

# Za prosledjeni broj dama, vraca listu parova (dama, pozicija), tako
# da je na svakoj vertikali table tacno jedna dama.
def generisi_pocetnu_konfiguraciju(broj_dama):
    konfiguracija = []
    for i in range(1, broj_dama + 1):
        par = (i, random.randint(1, broj_dama))
        konfiguracija.append(par)
    return konfiguracija

# Na osnovu prosledjene konfiguracije i tabu liste, generise se izmenjena
# okolina, koja je skupovna razlika okoline i tabu liste.
def generisi_okolinu(konfiguracija, tabu_lista):
    okolina = []
    n = len(konfiguracija)
    for i in range(n):
        okolina.extend([(i+1,polje) for polje in range(1,n+1) if konfiguracija[i][1] != polje])
    izmenjena_okolina = list(set(okolina) - set(tabu_lista))
    return izmenjena_okolina

def selektuj_najbolji_potez(okolina, konfiguracija):
    najbolji_potez = okolina[0]
    nova_konfiguracija = [(d, p) if d!=okolina[0][0] else okolina[0] for (d, p) in konfiguracija]
    min_prekrsaja = izracunaj_broj_prekrsaja(nova_konfiguracija)
    for i in range(1, len(okolina)):
        trenutni_potez = okolina[i]
        nova_konfiguracija = [(d, p) if d!=trenutni_potez[0] else trenutni_potez for (d, p) in konfiguracija]
        br_prekrsaja = izracunaj_broj_prekrsaja(nova_konfiguracija)
        if br_prekrsaja < min_prekrsaja:
            najbolji_potez = trenutni_potez
            min_prekrsaja = br_prekrsaja

    return najbolji_potez

# Vrsi tabu pretragu nad konfiguracijama.
def TabuDame(pocetna_konfiguracija, max_iteracija, velicina_tabu_liste):
    trenutna_konf = pocetna_konfiguracija
    najbolja_konf = pocetna_konfiguracija
    n = len(pocetna_konfiguracija)
    brojac = 0

    tabu_lista = []

    while brojac < max_iteracija and izracunaj_broj_prekrsaja(trenutna_konf) > 0:
        izmenjena_okolina = generisi_okolinu(trenutna_konf, tabu_lista)
        if len(izmenjena_okolina) > 0:
            potez = selektuj_najbolji_potez(izmenjena_okolina, trenutna_konf)
            tabu_lista.extend([(d,p) for d in trenutna_konf if d==potez[0]])
            trenutna_konf = [(d, p) if d!=potez[0] else potez for (d, p) in trenutna_konf]

        if izracunaj_broj_prekrsaja(trenutna_konf) < izracunaj_broj_prekrsaja(najbolja_konf):
            najbolja_konf = trenutna_konf

        if len(tabu_lista) > velicina_tabu_liste:
            tabu_lista.pop(0)

        brojac += 1

    return najbolja_konf


In [2]:
import timeit
from tqdm import tqdm
import pandas

In [15]:
def fitness(max_it, tabu_list_len):
    n = 20
    total_time = 0
    
    for it in range(10):
        start = timeit.default_timer()

        prime = n*n
        prime_konfiguracija = []

        for i in range(100):
            konfiguracija = generisi_pocetnu_konfiguraciju(n)

            nova_konfiguracija = TabuDame(konfiguracija, 100, 20)

            if (izracunaj_broj_prekrsaja(nova_konfiguracija) < prime):
                prime = izracunaj_broj_prekrsaja(nova_konfiguracija)
                prime_konfiguracija = nova_konfiguracija

                if prime == 0:
                    break

        stop = timeit.default_timer()

        total_time = total_time + stop - start
        
    avg_time = total_time/10
    return avg_time

In [12]:
def run_test():
    arr = []
    for it_no in tqdm(range(10, 100, 10)):
        for tabu_len in range(1, 25, 5):
            fit = fitness(it_no, tabu_len)
            arr.append([it_no, tabu_len, fit])
    return arr

In [17]:
arr = run_test()
df = pandas.DataFrame(arr, columns=['Max iterations', 'Length of tabu list', 'Average runtime for n=20'])
display(df)




  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [15:28<2:03:46, 928.28s/it]


 22%|██▏       | 2/9 [34:00<1:54:44, 983.55s/it]


 33%|███▎      | 3/9 [49:55<1:37:29, 974.99s/it]


 44%|████▍     | 4/9 [1:07:53<1:23:49, 1005.82s/it]


 56%|█████▌    | 5/9 [1:23:09<1:05:15, 978.80s/it] 


 67%|██████▋   | 6/9 [1:37:21<47:02, 940.96s/it]  


 78%|███████▊  | 7/9 [1:49:59<29:31, 885.97s/it]


 89%|████████▉ | 8/9 [2:05:03<14:51, 891.46s/it]


100%|██████████| 9/9 [2:15:11<00:00, 901.23s/it]


,Max iterations,Length of tabu list,Average runtime for n=20
0,10,1,2.213758
1,10,6,1.669315
2,10,11,1.647301
3,10,16,1.748306
4,10,21,2.004069
5,20,1,2.081054
6,20,6,2.171136
7,20,11,2.157285
8,20,16,2.071274
9,20,21,2.644559


In [20]:
df[df['Average runtime for n=20']==df['Average runtime for n=20'].min()]

,Max iterations,Length of tabu list,Average runtime for n=20
27,60,11,0.637598


In [22]:
df[df['Average runtime for n=20']<1.2]

,Max iterations,Length of tabu list,Average runtime for n=20
22,50,11,1.187593
23,50,16,0.961352
27,60,11,0.637598
31,70,6,1.090587
34,70,21,1.135153
38,80,16,0.805036
41,90,6,1.001105
42,90,11,1.108473
43,90,16,0.770418
